In [ ]:
##############################################################################
# What is PsyNeuLink?
#
# PsyNeuLink is an integrated language and toolkit for creating
# cognitive models. It decreases the overhead required for cognitive
# modeling by providing standard building blocks (DDMS, Neural Nets, etc.)
# and the means to connect them together in a single environment.
# PsyNeuLink is designed to make the user think about computation
# in a "mind/brain-like" way while imposing minimal constraint
# on the type of models that can be implemented.
##############################################################################

In [ ]:
##############################################################################
# How do I get PsyNeuLink?
# 
# Go to https://github.com/PrincetonUniversity/PsyNeuLink
# Press the green "Clone or download" button, then "Download ZIP"
# Unzip in the directory of your choice.
#
# Right now, you need explicit permission to download PsyNeuLink,
# so if you can't follow these directions, this is likely the reason.
#
# Once you have downloaded the repository, move this file to the main PsyNeuLink
# folder and restart the kernel from the jupyter toolbar (Kernel, Restart & Clear Output).
##############################################################################

In [ ]:
# Run this cell to download required packages.
! pip3 install toposort
! pip3 install mpi4py
! pip3 install numpy
! pip3 install matplotlib
! pip3 install typecheck-decorator

In [ ]:
# Making sure this tutorial is in the root PsyNeuLink directory,
# run this cell to install PsyNeuLink locally and upgrade to the
# current version. (Right now the tutorial is assuming version 0.0.1.dev1)
! pip3 install --upgrade .

In [ ]:
##############################################################################
# What will we do in this tutorial?
#
# This tutorial is meant to get you accustomed to the structure of PsyNeuLink
# and be able to construct basic models. Starting with a simple 1-to-1 transformation,
# we will build up to making the Stroop model from Cohen et al. (1990) and a 
# reinforcement learning agent
##############################################################################

In [3]:
# First we need to import the necessary packages. We will discuss their function
# as they are used

# This first package contains the basic methods to create and handle Processes.
from PsyNeuLink.Functions.Process import *

# The second set imports specific types of mechansims, a Transfer mechanism, that we 
# will be using in this tutorial.
from PsyNeuLink.Functions.Mechanisms.ProcessingMechanisms.Transfer import Transfer
from PsyNeuLink.Functions.Utilities.Utility import Linear, Logistic

# The third will handle projections
from PsyNeuLink.Functions.Projections.Mapping import Mapping

In [1]:
# Now we import tools for plotting so we can see what our mechanisms do.
import numpy as np
import matplotlib.pyplot as plt
% matplotlib inline

In [4]:
# Now we are ready to create our first mechanism. A mechanism is the basic unit
# of computation in PsyNeuLink. It takes inputs, runs them through a certain function, 
# then handles the outputs of that function. First we will make a Transfer mechanism 
# that performs a linear transformation on its input.

linear_transfer_mechanism = Transfer(function=Linear(slope = 2, intercept = 0))

# In this case, we didn't actually need to specify the slope and intercept as the 
# mechanism will default to reasonable values (in this case, 1 and 0 respectively).

In [5]:
# Now we need to put our mechanism in a process. In PsyNeuLink, a process is
# a collection of mechanisms and projections (which we will learn about later)
# connected in a certain pathway to be executed.
# Since we only have one mechanism, the pathway list has only one element.
# We will leave the other parameters to default.

linear_transfer_process = process(pathway = [linear_transfer_mechanism])

# Note: Make sure you do not run this cell multiple times. This would create multiple
# processes that share the same variable name in the PsyNeuLink registry causing
# unexpected behavior.

# We want to see how this mechanism is behaving, so we will turn on reporting
# just for this block. This happens at both the mechanism and process level
linear_transfer_mechanism.prefs.verbosePref = True
linear_transfer_mechanism.prefs.reportOutputPref = True
linear_transfer_process.prefs.verbosePref = True
linear_transfer_process.prefs.reportOutputPref = True

In [6]:
# Now we can run our process. Since our only mechanism is the 1-to-1 linear
# mechanism, the process output should be the same as its input. Run this cell
# a few times with different inputs to see how the output changes.
linear_transfer_process.execute([4])



****************************************


'Process-1' executing with:
- pathway: ['Transfer-1']
- input: 4

'Transfer-1' mechanism executed:
- input:   4.
- params:
	time_scale: TimeScale.TRIAL
	initial_state: 0
	function: Linear
		slope: 2
		intercept: 0
	noise: 0.0
- output:  8.

'Process-1' completed:
- output:  8.


****************************************



DeprecationWarning: DisplayFormatter._ipython_display_formatter_default is deprecated: use @default decorator instead.

In [ ]:
# Now let's see if it runs properly over a wide range of values
# Now we will want to get rid of some of that output. Again, we 
# need to set this for both our mechanism and our process

linear_transfer_mechanism.prefs.verbosePref = False
linear_transfer_mechanism.prefs.reportOutputPref = False
linear_transfer_process.prefs.verbosePref = False
linear_transfer_process.prefs.reportOutputPref = False

In [ ]:
# Finally, let's see the output of our process over a range of values using pyplot
xVals = np.linspace(-3, 3, num=51)
yVals = np.zeros((51,))
for i in range(xVals.shape[0]):
    yVals[i] = linear_transfer_process.execute([xVals[i]])[0]
    # Progress bar
    print("-", end="")
plt.plot(xVals, yVals)
plt.show()

In [ ]:
# Now let's put it all together and make a new logistic transfer process

# Create the mechanism
logistic_transfer_mechanism = Transfer(function=Logistic(gain = 1, bias = 0))

# Package into a process
logistic_transfer_process = process(pathway = [logistic_transfer_mechanism])

# Iterate and plot
xVals = np.linspace(-3, 3, num=51)
yVals = np.zeros((51,))
for i in range(xVals.shape[0]):
    yVals[i] = logistic_transfer_process.execute([xVals[i]])[0]
    # Progress bar
    print("-", end="")
plt.plot(xVals, yVals)
plt.show()

In [ ]:
# So far, all of our mechanisms have executed entirely on a single phase.
# Now we will make transfer units that integrate their input over time
# to work towards building elementary neural networks

linear_integrating_mechanism = Transfer(function=Linear(), noise = 0, rate = .1, time_scale = TimeScale.REAL_TIME)

In [ ]:
linear_integrating_process = process(pathway = [linear_integrating_mechanism])

In [ ]:
time_steps = np.linspace(0, 50, num=51)
yVals = np.zeros((51,))
for i in range(1, 51):
    yVals[i] = linear_integrating_process.execute([1])[0]
    # Progress bar
    print("-", end="")
plt.plot(time_steps, yVals)
plt.show()

# Notice that this time activation is accumulating slowly over time
# rather than all at once.

In [ ]:
# The final step before we make our first real model is to connect
# mechanisms together using projections. To start simple, we will
# connect two linear transfer mechanisms in series.
linear_input_unit = Transfer(function = Linear(slope = 2, intercept = 2))
linear_output_unit = Transfer(function = Linear(slope = 5, intercept = -4))
small_linear_process = process(pathway = [linear_input_unit, IDENTITY_MATRIX, linear_output_unit])

# IDENTITY_MATRIX is a keyword that provides a projection from the unit preceding
# it to the unit following that creates a one-to-one output to input projection
# between the two. Other useful projection keywords are...

In [ ]:
# Make sure we see the desired output
small_linear_process.execute([0])

In [ ]:
# Now let's make our projection definition a bit more explicit.
linear_input_unit = Transfer(function = Linear(slope = 2, intercept = 2))
linear_output_unit = Transfer(function = Linear(slope = 5, intercept = -4))

# This time we create a Mapping projection which takes an output state and sends
# its value multiplied by a given matrix to an input state (instead of explicitly
# calling the output and input states, we here just provide the sending and receiving
# mechanisms for simplicity).
unit_mapping_projection = Mapping(sender = linear_input_unit, receiver = linear_output_unit, matrix = np.asarray([[1]]))
small_linear_process = process(pathway = [linear_input_unit, unit_mapping_projection, linear_output_unit])

In [ ]:
# Change the value in the projection matrix above and make sure the output
# matches your expectations
print(small_linear_process.execute([0]))
xVals = np.linspace(-3, 3, num=51)
yVals = np.zeros((51,))
for i in range(xVals.shape[0]):
    yVals[i] = small_linear_process.execute([xVals[i]])[0]
    # Progress bar
    print("-", end="")
plt.plot(xVals, yVals)
plt.show()

In [ ]:
#
linear_input_unit = Transfer(function = Linear(), noise = 0, rate = 0.1, time_scale = TimeScale.REAL_TIME)
logistic_input_unit = Transfer(function = Logistic(), noise = 0, rate = 0.1, time_scale = TimeScale.REAL_TIME)
linear_output_unit = Transfer(function = Linear(), noise = 0, rate = 0.1, time_scale = TimeScale.REAL_TIME)

projection_1 = Mapping(sender = linear_input_unit, receiver = linear_output_unit, matrix = np.asarray([[1]]))
projection_2 = Mapping(sender = logistic_input_unit, receiver = linear_output_unit, matrix = np.asarray([[-1]]))

small_dynamic_process = process(pathway = [linear_input_unit, logistic_input_unit, linear_output_unit])

time_steps = np.linspace(0, 50, num=51)
yVals = np.zeros((51,))
for i in range(1, 51):
    yVals[i] = small_dynamic_process.execute([1, 1])[0]
    # Progress bar
    print("-", end="")
plt.plot(time_steps, yVals)
plt.show()